A **rigid transformation** (also called Euclidean transformation or Euclidean isometry) - a geometric transformation of a Euclidean space that **preserves the Euclidean distance** between every pair of points. PyTorch3D provides a high-level encapsulation of these transformations in the "pytorch3d.transforms.Transform3d" class.

- Lie group **SO(3)** - special orthogonal group in 3D (informally speaking,the set of all the rotation transformations);
- Lie group **SE(3)** - special Euclidean group in 3D (informally, the set of all the rigid transformations in 3D).

In [1]:
# Importing the packages
import torch
from pytorch3d.transforms.so3 import so3_exp_map, so3_log_map, hat_inv, hat

In [2]:
# Defining PyTorch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be more slowly")

In [3]:
# Defining a mini-batch of four rotations

# Each rotation is represented by one 3D vector. 
# The direction of the vector represents the rotation axis 
# and the amplitude of the vector represents the angle of rotation:

log_rot = torch.zeros([4, 3], device = device) # batch size is 4 and each rotation is a 3D vector. 

log_rot[0, 0] = 0.001
log_rot[0, 1] = 0.0001
log_rot[0, 2] = 0.0002

log_rot[1, 0] = 0.0001
log_rot[1, 1] = 0.001
log_rot[1, 2] = 0.0002

log_rot[2, 0] = 0.0001
log_rot[2, 1] = 0.0002
log_rot[2, 2] = 0.001

log_rot[3, 0] = 0.001
log_rot[3, 1] = 0.002
log_rot[3, 2] = 0.003

log_rot

tensor([[1.0000e-03, 1.0000e-04, 2.0000e-04],
        [1.0000e-04, 1.0000e-03, 2.0000e-04],
        [1.0000e-04, 2.0000e-04, 1.0000e-03],
        [1.0000e-03, 2.0000e-03, 3.0000e-03]])

In [4]:
# Using hat operator, to convert rotations into the 3 x 3 skew-symmetric matrix 
log_rot_hat = hat(log_rot)

print('log_rot_hat shape = ', log_rot_hat.shape)
print('log_rot_hat = ', log_rot_hat)

log_rot_hat shape =  torch.Size([4, 3, 3])
log_rot_hat =  tensor([[[ 0.0000e+00, -2.0000e-04,  1.0000e-04],
         [ 2.0000e-04,  0.0000e+00, -1.0000e-03],
         [-1.0000e-04,  1.0000e-03,  0.0000e+00]],

        [[ 0.0000e+00, -2.0000e-04,  1.0000e-03],
         [ 2.0000e-04,  0.0000e+00, -1.0000e-04],
         [-1.0000e-03,  1.0000e-04,  0.0000e+00]],

        [[ 0.0000e+00, -1.0000e-03,  2.0000e-04],
         [ 1.0000e-03,  0.0000e+00, -1.0000e-04],
         [-2.0000e-04,  1.0000e-04,  0.0000e+00]],

        [[ 0.0000e+00, -3.0000e-03,  2.0000e-03],
         [ 3.0000e-03,  0.0000e+00, -1.0000e-03],
         [-2.0000e-03,  1.0000e-03,  0.0000e+00]]])


In [5]:
# The backward conversion, if needed
log_rot_copy = hat_inv(log_rot_hat)

print('log_rot_copy shape = ', log_rot_copy.shape)
print('log_rot_copy = ', log_rot_copy)

log_rot_copy shape =  torch.Size([4, 3])
log_rot_copy =  tensor([[1.0000e-03, 1.0000e-04, 2.0000e-04],
        [1.0000e-04, 1.0000e-03, 2.0000e-04],
        [1.0000e-04, 2.0000e-04, 1.0000e-03],
        [1.0000e-03, 2.0000e-03, 3.0000e-03]])


In [6]:
# From the gradient matrix, we can compute the rotation matrix - 
# so3_exp_map

rotation_matrices = so3_exp_map(log_rot)
print('rotation_matrices = ', rotation_matrices)

rotation_matrices =  tensor([[[ 1.0000e+00, -1.9995e-04,  1.0010e-04],
         [ 2.0005e-04,  1.0000e+00, -9.9997e-04],
         [-9.9898e-05,  9.9999e-04,  1.0000e+00]],

        [[ 1.0000e+00, -1.9995e-04,  9.9999e-04],
         [ 2.0005e-04,  1.0000e+00, -9.9898e-05],
         [-9.9997e-04,  1.0010e-04,  1.0000e+00]],

        [[ 1.0000e+00, -9.9997e-04,  2.0005e-04],
         [ 9.9999e-04,  1.0000e+00, -9.9898e-05],
         [-1.9995e-04,  1.0010e-04,  1.0000e+00]],

        [[ 9.9999e-01, -2.9989e-03,  2.0015e-03],
         [ 3.0010e-03,  9.9999e-01, -9.9698e-04],
         [-1.9985e-03,  1.0030e-03,  1.0000e+00]]])


In [7]:
# inverse conversation of the rotation matrix to the gradient matrix - 
# so3_log_map
log_rot_again = so3_log_map(rotation_matrices)
print('log_rot_again = ', log_rot_again)

log_rot_again =  tensor([[9.9999e-04, 9.9999e-05, 2.0000e-04],
        [9.9999e-05, 9.9999e-04, 2.0000e-04],
        [9.9999e-05, 2.0000e-04, 9.9999e-04],
        [9.9999e-04, 2.0000e-03, 3.0000e-03]])
